## Denovo design of technetium labeled proteins-Prediction of alphafold2
This script details the execution of Alphafold2 step after initial proteinMPNN runs. It also lists potential analysis to screen designed proteins. This script should be executed after installing the dependencies in the "mlfold.yml" file.

In [ ]:
import os, sys, glob
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import getpass
import subprocess
import time
import importlib
from shutil import copy2

%load_ext autoreload
%autoreload 2

### Path to this cloned GitHub repo:
SCRIPT_DIR = os.getcwd()
assert os.path.exists(SCRIPT_DIR)
sys.path.append(SCRIPT_DIR+"/scripts/utils")
import utils

In [ ]:
os.chdir(WDIR)

AF2_DIR = f"{WDIR}/2_af2"
os.makedirs(AF2_DIR, exist_ok=True)
os.chdir(AF2_DIR)

### First collecting MPNN outputs and creating FASTA files for AF2 input
mpnn_fasta = utils.parse_fasta_files(glob.glob(f"{MPNN_DIR}/seqs/*.fa"))
mpnn_fasta = {k: seq.strip() for k, seq in mpnn_fasta.items() if "model_path" not in k}  # excluding the diffused poly-A sequence
# Giving sequences unique names based on input PDB name, temperature, and sequence identifier
mpnn_fasta = {k.split(",")[0]+"_"+k.split(",")[2].replace(" T=", "T")+"_0_"+k.split(",")[1].replace(" id=", ""): seq for k, seq in mpnn_fasta.items()}

print(f"A total on {len(mpnn_fasta)} sequences will be predicted.")

## Splitting the MPNN sequences based on length
## Use group size of >40 when running on GPU. Also depends on how many sequences and resources you have.

SEQUENCES_PER_AF2_JOB = 5  # CPU
if USE_GPU_for_AF2 is True:
    SEQUENCES_PER_AF2_JOB = 100  # GPU
mpnn_fasta_split = utils.split_fasta_based_on_length(mpnn_fasta, SEQUENCES_PER_AF2_JOB, write_files=True)

In [ ]:
import subprocess
import glob
import os

## === Setup AlphaFold2 parameters ===
AF2_recycles = 3
AF2_models = "4"  # Add more like "3 4 5" if needed

commands_af2 = []
cmds_filename_af2 = "commands_af2"

# Write all commands to file for reference
with open(cmds_filename_af2, "w") as file:
    for ff in glob.glob("*.fasta"):
        cmd = (
            f"{PYTHON['af2']} {AF2_script} "
            f"--af-nrecycles {AF2_recycles} --af-models {AF2_models} "
            f"--fasta {ff} --scorefile {ff.replace('.fasta', '.csv')}"
        )
        commands_af2.append(cmd)
        file.write(cmd + "\n")

print("Example AF2 command:")
print(commands_af2[-1])

## === Run AlphaFold2 commands locally, one by one ===
for cmd in commands_af2:
    print(f"\nRunning locally:\n{cmd}\n")
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("STDOUT:\n", result.stdout.decode())
    print("STDERR:\n", result.stderr.decode())

In [ ]:
## If you're done with diffusion and happy with the outputs then mark it as done
AF2_DIR = f"{WDIR}/2_af2"
os.chdir(AF2_DIR)

if not os.path.exists(AF2_DIR+"/.done"):
    with open(f"{AF2_DIR}/.done", "w") as file:
        file.write(f"Run user: {username}\n")

## Analyzing alphafold2 predictions

Before proceeding, check if there are any zero vector atoms which could interfere with the analyses.

In [ ]:
from Bio.PDB import PDBParser
import os

folder = "/home/projects/protein_design/binder_diffusion/2_af2/"
parser = PDBParser(QUIET=True)

for fname in os.listdir(folder):
    if not fname.endswith(".pdb"):
        continue
    path = os.path.join(folder, fname)
    try:
        structure = parser.get_structure("ref", path)
        zero_coords = False
        for atom in structure.get_atoms():
            if all(abs(coord) < 1e-5 for coord in atom.get_coord()):
                zero_coords = True
                break
        if zero_coords:
            print(f"{fname} contains zero-vector atoms")
    except Exception as e:
        print(f"{fname} failed to parse: {e}")

In [ ]:
#This code removes side chain atoms with 0.0 coordinatates. Modify to ensure motif residues are unaffected
# Directory containing your PDB files
pdb_dir = "/home/projects/protein_design/binder_diffusion/0_diffusion/filtered_structures/"

# Go through all files in the directory
for file_name in os.listdir(pdb_dir):
    if file_name.lower().endswith(".pdb"):
        file_path = os.path.join(pdb_dir, file_name)
        new_lines = []

        with open(file_path, 'r') as infile:
            for line in infile:
                if line.startswith(("ATOM", "HETATM")) and len(line) >= 54:
                    try:
                        x = float(line[30:38])
                        y = float(line[38:46])
                        z = float(line[46:54])
                        if x == 0.0 and y == 0.0 and z == 0.0:
                            continue  # Skip this line
                    except ValueError:
                        pass  # In case of parsing error, just keep the line
                new_lines.append(line)

        # Overwrite original file with cleaned content
        with open(file_path, 'w') as outfile:
            outfile.writelines(new_lines)

        print(f"Cleaned: {file_name}")

In [ ]:
# Combining all CSV scorefiles into one
AF2_DIR = f"{WDIR}/2_af2"
DIFFUSION_DIR = f"{WDIR}/0_diffusion"

os.system("head -n 1 $(ls *aa*.csv | shuf -n 1) > scores.csv ; for f in *aa*.csv ; do tail -n +2 ${f} >> scores.csv ; done")
assert os.path.exists("scores.csv"), "Could not combine scorefiles"

### Calculating the RMSDs of AF2 predictions relative to the diffusion outputs
### Catalytic residue sidechain RMSDs are calculated in the reference PDB has REMARK 666 line present

analysis_cmd = f"{PYTHON['general']} {SCRIPT_DIR}/scripts/utils/analyze_af2.py --scorefile scores.csv "\
               f"--ref_path {DIFFUSION_DIR}/filtered_structures --mpnn --params {' '.join(params)}"


if len(glob.glob(f"{AF2_DIR}/*.pdb")) > 100:
    ## Analyzing locally
    p = subprocess.Popen(analysis_cmd, shell=True)
    (output, err) = p.communicate()
else:
    ## Running as a Slurm job
    submit_script = "submit_af2_analysis.sh"
    utils.create_slurm_submit_script(filename=submit_script, name="af2_analysis",
                                     mem="16g", N_cores=8, time="0:20:00", email=EMAIL,
                                     command=analysis_cmd, outfile_name="output_analysis")

    p = subprocess.Popen(["sbatch", submit_script])
    (output, err) = p.communicate()

In [ ]:
## This plots out the model ouput
## There is a seperate notebook for a more visually appealing plot used for my poster
scores_af2 = pd.read_csv("scores.sc", sep="\s+", header=0)

### Filtering AF2 scores based on lddt and rmsd
# Define your desired cutoffs here:
AF2_filters = {"lDDT": [80.0, ">="],
               "rmsd": [2.0, "<="]}  # 1st catalytic residue sc-rmsd

scores_af2_filtered = utils.filter_scores(scores_af2, AF2_filters)
utils.dump_scorefile(scores_af2_filtered, "filtered_scores.sc")

## Plotting AF2 scores
plt.figure(figsize=(12, 3))
for i,k in enumerate(AF2_filters):
    plt.subplot(1, 3, i+1)
    plt.hist(scores_af2[k])
    plt.title(k)
    plt.xlabel(k)
plt.tight_layout()
plt.show()
    
utils.plot_score_pairs(scores_af2, "lDDT", "rmsd", AF2_filters["lDDT"][0], AF2_filters["rmsd"][0])

In [ ]:
### Copying good predictions to a separate directory
## This step is important to add hydrogens and have a complete structure. However pyrosetta tries to add an amino acid as the ligand instead of technetium. Manually remove this
## Superimpose the designs from this step with the generated backbone to have the tc complex back in the structure 
os.chdir(AF2_DIR)

if len(scores_af2_filtered) > 0:
    os.makedirs("good", exist_ok=True)
    good_af2_models = [row["Output_PDB"]+".pdb" for idx,row in scores_af2_filtered.iterrows()]
    for pdb in good_af2_models:
        copy2(pdb, f"good/{pdb}")
    good_af2_models = glob.glob(f"{AF2_DIR}/good/*.pdb")
else:
    sys.exit("No good models to continue this pipeline with")

os.chdir(f"{AF2_DIR}/good")


### Aligning the ligand back into the AF2 predictions.
### This is done by aligning the AF2 model to diffusion output and copying over the ligand using PyRosetta.
### --fix_catres option will readjust the rotamer and tautomer of 
### any catalytic residue to be the same as in the reference model.

align_cmd = f"{PYTHON['general']} {SCRIPT_DIR}/scripts/utils/place_ligand_after_af2.py "\
            f"--outdir with_heme2 --params {' '.join(params)} --fix_catres "\
            f"--pdb {' '.join(good_af2_models)} "\
            f"--ref {' '.join(glob.glob(DIFFUSION_DIR+'/filtered_structures/*.pdb'))}"

p = subprocess.Popen(align_cmd, shell=True)
(output, err) = p.communicate()

In [ ]:
##END Subs